In [1]:
from dataset import create_dataloaders

train_loader = create_dataloaders("../data/Training/01.원천데이터", batch_size=8, test_ratio = 0, image_size=160, workers=16)
val_loader = create_dataloaders( "../data/Validation/01.원천데이터", batch_size=8, test_ratio = 0, image_size=160, workers=16)

In [2]:
import torch
from vision_model import R2Plus1DNet
from sensor_model import FallDetection1DCNN

DEVICE = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {DEVICE} device")

vision_checkpoint = torch.load("vision_24_12_13.pth", map_location=DEVICE)
vision_model = R2Plus1DNet(num_classes=1).to(DEVICE)
vision_model.load_state_dict(vision_checkpoint['model_state_dict'], strict=False)

sensor_checkpoint = torch.load("sensor_24_12_15.pth", map_location=DEVICE)
sensor_model = FallDetection1DCNN(num_classes=1).to(DEVICE)
sensor_model.load_state_dict(sensor_checkpoint, strict=False)

Using cuda device


_IncompatibleKeys(missing_keys=[], unexpected_keys=['fc.weight', 'fc.bias'])

In [3]:
import torch.nn as nn

class MultiModalNet(nn.Module):
    def __init__(self, num_classes=10):
        super(MultiModalNet, self).__init__()
        self.video_net = vision_model
        self.sensor_net = sensor_model

        self.fc_sequence = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, video_x, sensor_x):
        video_feat = self.video_net(video_x)
        sensor_feat = self.sensor_net(sensor_x)


        combined = torch.cat([video_feat, sensor_feat], dim=1)

        output = self.fc_sequence(combined)
        return output


In [4]:
model = MultiModalNet(num_classes=1).to(DEVICE)

In [5]:
import torch.optim as optim

learning_rate=0.0001 
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, f1_score

def val(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    all_labels = []
    all_preds = []

    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc="Validating", leave=False)
        for video, sensor, label in progress_bar:
            video = video.to(DEVICE)
            sensor = sensor.to(DEVICE)
            label = label.float().unsqueeze(1).to(DEVICE) 

            output = model(video, sensor)

            loss = criterion(output, label)
            val_loss += loss.item()

            predicted = (torch.sigmoid(output) > 0.5).float()
            correct_predictions += (predicted == label).sum().item()
            total_samples += label.size(0)

            all_labels.extend(label.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    avg_loss = val_loss / len(val_loader)
    accuracy = correct_predictions / total_samples * 100

    f1 = f1_score(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)

    print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%, F1 Score: {f1:.2f}")
    print("Confusion Matrix:\n", cm)

    return avg_loss, accuracy

In [7]:
def train(model, train_loader, val_loader, num_epochs):
    best_val_acc = 0

    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=False)

        for video, sensor, label in progress_bar:
            video = video.to(DEVICE)
            sensor = sensor.to(DEVICE)
            label = label.float().unsqueeze(1).to(DEVICE)

            optimizer.zero_grad()

            output = model(video, sensor)

            loss = criterion(output, label)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            predicted = (torch.sigmoid(output) > 0.5).float()
            correct_predictions += (predicted == label).sum().item()
            total_samples += label.size(0)

            progress_bar.set_postfix(loss=running_loss / (progress_bar.n + 1))

        accuracy = correct_predictions / total_samples * 100
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

        avg_val_loss, val_acc = val(model, val_loader, criterion)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            model_save_path = './model.pth'
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_val_loss
            }, model_save_path)
            print(f"Model and parameters saved to {model_save_path}")

num_epochs = 10
train(model, train_loader, val_loader, num_epochs=num_epochs)


Epoch [1/10], Loss: 0.1009, Accuracy: 96.62%


Validation Loss: 0.0512, Accuracy: 98.64%, F1 Score: 0.99
Confusion Matrix:
 [[ 566    2]
 [  29 1675]]
Model and parameters saved to ./model.pth


Epoch [2/10], Loss: 0.0277, Accuracy: 99.22%


Validation Loss: 0.0306, Accuracy: 99.38%, F1 Score: 1.00
Confusion Matrix:
 [[ 564    4]
 [  10 1694]]
Model and parameters saved to ./model.pth


Epoch [3/10], Loss: 0.0150, Accuracy: 99.59%


...

Validation Loss: 0.0504, Accuracy: 99.25%, F1 Score: 1.00
Confusion Matrix:
 [[ 560    8]
 [   9 1695]]


Epoch [9/10], Loss: 0.0050, Accuracy: 99.84%


Validation Loss: 0.0550, Accuracy: 99.30%, F1 Score: 1.00
Confusion Matrix:
 [[ 560    8]
 [   8 1696]]


Epoch [10/10], Loss: 0.0023, Accuracy: 99.93%


Validation Loss: 0.0494, Accuracy: 99.43%, F1 Score: 1.00
Confusion Matrix:
 [[ 566    2]
 [  11 1693]]
Model and parameters saved to ./model.pth
